In [ ]:
import pandas as pd

In [ ]:
data_df = pd.read_csv("../data/original_data.csv")

In [ ]:
data_df.duplicated().sum()

In [ ]:
data_df = data_df[data_df['Description'].notna()].reset_index(drop=True)

In [ ]:
# Split Category strings into lists (split on comma, strip whitespace)
def parse_categories(val):
    if pd.isna(val):
        return []
    if isinstance(val, str):
        parts = [p.strip() for p in val.split(',') if p.strip()]
        return parts
    if isinstance(val, list):
        return val
    return [str(val)]

data_df['Category'] = data_df['Category'].apply(parse_categories)

In [ ]:
data_df

In [ ]:
data_df['Authors'] = data_df['Authors'].apply(lambda x: x.replace("By ", "") if pd.notna(x) else x)

In [ ]:
# Filter out rows with invalid author formats (all uppercase, no commas)
def is_valid_author_format(val):
    if pd.isna(val):
        return True  # Keep NaN values for now

    val_str = str(val).strip()

    # Check if string is all uppercase (likely invalid format)
    if val_str.isupper():
        return False

    # Must contain at least one comma for proper "Lastname, Firstname" format
    if ',' not in val_str:
        return False

    return True

data_df = data_df[data_df['Authors'].apply(is_valid_author_format)].reset_index(drop=True)

In [ ]:
import re

def parse_authors(val):
    if pd.isna(val):
        return []

    # First split by " and " to separate multiple authors
    authors = re.split(r'\s+and\s+', str(val))

    # Then split each author segment by comma
    all_authors = []
    for author in authors:
        parts = author.split(',')
        parts = [name.strip() for name in parts if name.strip()]
        all_authors.extend(parts)

    # Filter out parts that are ONLY known titles/roles or contain museum/institution keywords
    # Common academic/editorial roles: EDT, COR, ILT, TRN, FRW, etc.
    filtered_authors = []
    known_titles = ['EDT', 'COR', 'ILT', 'TRN', 'FRW', 'PHD', 'DR', 'MD', 'PROF', 'ESQ']
    institution_keywords = ['MUSEUM', 'LIBRARY', 'SOCIETY', 'INSTITUTE', 'FOUNDATION', 'ASSOCIATION', 'UNIVERSITY']

    for author in all_authors:
        stripped = author.strip()
        # Remove parentheses and dots for comparison
        cleaned = re.sub(r'[\(\)\.]', '', stripped).upper().strip()

        # Skip only if it exactly matches a known title
        if cleaned in known_titles:
            continue

        # Skip if it contains institution keywords
        if any(keyword in stripped.upper() for keyword in institution_keywords):
            continue

        # Remove parentheses and their contents from the author name
        cleaned_author = re.sub(r'\s*\([^)]*\)', '', author).strip()
        if cleaned_author:  # Only add if there's something left after removing parentheses
            filtered_authors.append(cleaned_author)

    # Join every two consecutive elements to form full names (Lastname + Firstname)
    full_names = []
    for i in range(0, len(filtered_authors), 2):
        if i + 1 < len(filtered_authors):
            # Join lastname and firstname
            full_names.append(f"{filtered_authors[i]}, {filtered_authors[i+1]}")
        else:
            # Odd number of parts, keep the last one as is
            full_names.append(filtered_authors[i])

    return full_names

In [ ]:
data_df['Authors'] = data_df['Authors'].apply(parse_authors)

In [ ]:
data_df.to_csv("../data/processed_data.csv", index=False)

In [ ]:
data_df = pd.read_csv("/content/processed_data.csv")

Checking for incomplete or too short descriptions.

In [ ]:
import re
import pandas as pd

def is_bad_description(text):
    # Not a string or blank
    if not isinstance(text, str) or not text.strip():
        return True

    cleaned = text.strip()

    # 1. Exactly "#NAME?" or similar Excel errors
    if cleaned.upper() in ["#NAME?", "#REF!", "#N/A", "#NA", "#NULL!", "#DIV/0!"]:
        return True

    # 2. Contains any hash "#..."
    if "##" in cleaned:
        return True

    # 3. Description is basically punctuation only
    if cleaned in [".", "..", "...", "-", "--", "_"]:
        return True

    # 4. Fewer than 5 words
    if len(cleaned.split()) < 5:
        return True

    return False


In [ ]:
bad_desc_mask = data_df['Description'].apply(is_bad_description)
bad_descriptions_df = data_df[bad_desc_mask].copy()
len(bad_descriptions_df)

In [ ]:
bad_descriptions_df[['Title', 'Authors', 'Description']].head(28)

In [ ]:
# Manual fix of missing book descriptions
# 1) Dictionary mapping the specific index to the new description
manual_desc_updates = {

    7780: "A practical guide to coping with illness and medical challenges, this book explores the emotional and psychological obstacles people face as they work toward recovery and resilience.",
    8417: "An educational resource designed to help children understand their role in caring for the planet, offering accessible activities and lessons that build environmental awareness.",
    9341: "A historical study of Confederate military strategy, examining the campaigns led by Robert E. Lee and Stonewall Jackson during the American Civil War.",
    12142: "A visual reference for gardeners featuring new and creative ideas for using annuals. Includes colorful plant suggestions, design tips, and techniques for adding seasonal impact to the home landscape.",
    18040: "A hands-on guide to planning and building fences, walls, and gates for outdoor spaces. Provides step-by-step instructions, construction advice, and design inspiration for homeowners.",
    19447: "A collection of simple and nutritious recipes aimed at reducing dietary fat. The book emphasizes healthy ingredients, practical cooking methods, and everyday meals for improving overall wellness.",
    21680: "A small gift book celebrating the permanence of love, featuring inspirational quotations and reflections intended to uplift and encourage long-term relationships.",
    22046: "An overview of a diet program centered on a specific nutritional bean extract. The authors explain how the plan supports energy, weight management, and improved metabolic health.",
    26318: "A craft and decorative guide filled with ideas for creating handmade Christmas tree ornaments and festive holiday decorations.",
    28827: "A historical survey exploring the role of sexuality in human societies, drawing on research, cultural analysis, and case studies from ancient to modern times.",
    28915: "A humorous collection of quotations and observations about fatherhood, compiled to celebrate and gently tease dads of all ages.",
    33211: "A bright and simple early-learning board book introducing young children to colors through clear images and easy-to-recognize objects.",
    35391: "A curated collection of memorable and motivational quotations about golf, drawing inspiration from players, writers, and enthusiasts of the sport.",
    35943: "A beginner-friendly mystery story for young readers in which a child detective solves a spooky case at a haunted hotel, uncovering clues and learning problem-solving skills along the way.",
    38871: "A small gift book celebrating the bond between grandmothers and grandchildren, filled with warm, loving quotations suitable for sharing and gifting.",
    39126: "A themed collection of quotations reflecting the beauty, spirit, and companionship associated with horses, ideal for equestrians and gift-giving occasions.",
    41060: "A craft book based on the American Girl Samantha series, featuring historical craft projects inspired by turn-of-the-century household traditions and activities.",
    44181: "A suspense novel in the Lucas series involving a private investigator drawn into a dangerous case. The story blends crime, tension, and personal stakes.",
    44610: "A gift-book collection of heartfelt quotations about the unique and lasting bond between mothers and sons.",
    46144: "A professional reference guide summarizing the most commonly used psychotropic medications. Includes dosage information, clinical applications, side effects, and treatment guidance.",
    49614: "A large activity book for children filled with creative projects, puzzles, coloring pages, and hands-on exercises designed to encourage imaginative play.",
    51463: "A quilting guide from a small-town quilt shop, offering patterns, practical techniques, and inspiration for both new and experienced quilters.",
    57814: "An early-learning concept book that introduces young children to the idea of time using simple explanations, everyday examples, and engaging photographs.",
    62154: "A lighthearted collection of computer-themed jokes and cartoons aimed at readers who enjoy technology-related humor.",
    62520: "A concise introduction to the ancient Maya civilization, highlighting its history, cultural achievements, and archaeological discoveries.",
    62582: "A collection of inspirational quotations celebrating the special relationship shared between sisters, intended for reflection or gift-giving.",
    62762: "A tactile board book for babies featuring animals found in the zoo. Designed to encourage sensory exploration through simple text and touch-and-feel elements.",
    64489: "A simple and reassuring picture book for young children about what firefighters do, introducing their equipment, duties, and role in helping the community."

}
# 2) Original dataframe update
for index, new_desc in manual_desc_updates.items():
    if index in data_df.index:
        data_df.loc[index, 'Description'] = new_desc

Language detection

In [ ]:
!pip install langdetect pandas tqdm

In [ ]:
from langdetect import detect, detect_langs, LangDetectException
from tqdm.auto import tqdm

tqdm.pandas()

In [ ]:
def detect_language(text):
    """
    Returns (lang, prob) for a given text or (None, None) if detection fails.
    """
    if not isinstance(text, str) or not text.strip():
        return None, None

    # Very short texts tend to give random results
    if len(text) < 20:
        return None, None

    try:
        # detect_langs returns a list
        langs = detect_langs(text)
        top = langs[0]
        lang = top.lang
        prob = top.prob
        return lang, prob
    except LangDetectException:
        return None, None


In [ ]:
data_lang = data_df.copy()

In [ ]:
# Detection of language
data_lang[['lang', 'lang_conf']] = data_lang['Description'].progress_apply(
    lambda x: pd.Series(detect_language(x))
)

In [ ]:
# Counts per language
print(data_lang['lang'].value_counts(dropna=False))

In [ ]:
# Non english texts detected
non_en = data_lang[(data_lang['lang'] != 'en') & (data_lang['lang_conf'] > 0.0)]
non_en[['Title', 'Authors', 'Description', 'lang', 'lang_conf']].head(5)

In [ ]:
manual_desc_updates_lang = {
    152: ( "A devotional book that explains the unchanging love of God and His desire "
        "to bless His children. Hinn explores how obedience, faith, and biblical "
        "giving open the way to experiencing God’s promises in everyday life."
    ),
    4462: ( "In this eleventh Father Koesler mystery, a small new monastery in Detroit "
        "suddenly attracts crowds and large donations after a supposed miracle gives "
        "sight to a blind woman. Asked to look into the situation, Father Koesler "
        "must determine whether the community’s charismatic leader is a genuine "
        "man of God or at the center of a dangerous deception."
    ),
    4654: ( "Swindoll challenges readers to drop their emotional defenses and remove the "
        "masks they wear in relationships. Through biblical insight and practical "
        "examples, he shows how authenticity, honesty, and vulnerability can heal "
        "old hurts and lead to deeper, more meaningful connections with others."
    ),
    5624: ( "An insider’s guide to modern espionage gear, explaining how listening "
        "devices, surveillance cameras, alarm systems, explosives detectors, safes, "
        "protective clothing, weapons, disguises, and other equipment are used. "
        "The authors also point readers to sources for specialized spy-tech tools."
    ),
    8769: ("A health guide built around traditional Latin American foods and remedies. "
        "Drawing on nutrition research and medical experts, it highlights everyday "
        "ingredients that may help prevent or ease common conditions such as diabetes, "
        "high cholesterol, high blood pressure, and excess weight, while offering "
        "practical advice and recipes for healthier eating."
    ),
    9147: ("A men’s devotional about the strength that comes from shared faith. Wagner "
        "explores five foundational truths that help men build deep friendships and "
        "supportive communities rooted in a common commitment to Jesus Christ."
    ),
    11068: ( "Using the Old Testament story of Nehemiah, Swindoll draws out principles of "
        "effective leadership at work, in ministry, and at home. He shows how wise "
        "leaders handle motivation, discouragement, opposition, and adversity with "
        "integrity while keeping their vision focused on God’s purposes."
    ),
    11183: (
        "A short, practical guide that offers simple ideas for talking with children "
        "about God. Temple suggests conversational, everyday ways to introduce faith, "
        "answer questions, and make discussion about spiritual things feel natural "
        "and open in the home."
    ),

    11435: ( "Journalist María Antonieta Collins reflects on her own turbulent encounters "
        "with ex-husbands, ex-wives, and ex-partners, especially when her husband’s "
        "former spouse suddenly reappears and disrupts their marriage. Blending "
        "memoir, humor, and advice, she offers guidance on how to set boundaries, "
        "protect one’s heart, and make wise decisions when past relationships refuse "
        "to stay in the past."
    ),
    13384: ( "A diver’s guide to Hawaii’s underwater world, describing colorful coral "
        "reefs, dramatic lava tunnels and pinnacles, and the many unique species "
        "found at popular dive sites throughout the islands."
    ),
    13937: ( "In this memoir, news anchor María Elena Salinas recounts her rise from a "
        "working-class Los Angeles neighborhood to becoming one of the most recognized "
        "Spanish-language journalists in the United States. Along the way she confronts "
        "family secrets—especially the revelation that her beloved father had once been "
        "a Catholic priest—and reflects on identity, faith, career, and the cost of truth."
    ),
    16562: ("A near-future science-fiction novel told through e-mail exchanges between a "
        "grad student and an experimental artificial intelligence called EDGAR. As the "
        "AI grows more self-aware, their relationship draws the attention of government "
        "agencies and raises unsettling questions about consciousness, control, and the "
        "moral limits of machine intelligence."
    ),
    25816: ("A compact Precious Moments gift Bible featuring angel-themed artwork by Sam "
        "Butcher. It includes the full Reina-Valera 1960 text, full-color illustrations, "
        "presentation pages, and space to record personal reflections, making it a "
        "keepsake for those who love these classic characters."
    ),
    26115: ("A Spanish-language edition of the classic picture book Goodnight Moon. In a "
        "quiet green room, a little bunny says goodnight to everything around him— "
        "from the moon and the stars to mittens and kittens—creating a soothing, "
        "rhythmic bedtime story for young children and adults to share."
    ),
    39813: ( "A detailed narrative history of the Battle of Gettysburg, guiding readers "
        "step by step through the three days of fighting. Stackpole combines maps, "
        "photographs, and eyewitness accounts to explain the movements of the armies, "
        "the decisions of their commanders, and the experiences of soldiers on both sides."
    ),
    40850: ("The second level of a standards-based English course for adult beginners. "
        "Stand Out 2 integrates language development with real-life projects, listening "
        "and pronunciation practice, grammar in context, and supplemental exercises, "
        "helping learners communicate at work, at home, and in the community."
    ),
    42424: ( "Drawing on his experience with men’s ministry, Patrick Morley explains how "
        "men think, what they struggle with, and what they often wish they could say "
        "to their wives. The book helps women better understand the spiritual, "
        "emotional, and practical needs of their husbands so they can strengthen "
        "their marriages."
    ),

    42581: ("A devotional based on the brief Old Testament prayer of Jabez in 1 Chronicles 4:10. "
        "Wilkinson explores how asking God to bless, enlarge one’s territory, and keep "
        "harm away can reshape a believer’s expectations and open doors to a life of "
        "greater fruitfulness and service."
    ),

    45093: ("A novelist’s wife disappears without explanation, leaving him obsessed with "
        "finding her and with the idea that she has become his ‘Zahir’—a consuming "
        "fixation. His search leads from the comfortable world of Paris to distant "
        "Kazakhstan, where he begins to question success, freedom, marriage, and the "
        "true nature of love."
    ),
    46809: "Alexander wakes up to a piece of gum stuck in his hair and soon discovers that everything about this day will be terrible. From losing his best friend to unpleasant surprises at school and dinner, Alexander’s string of misfortunes makes for a humorous and relatable story about handling bad days.",
    50761: "A standards-based English course designed for adult beginners. The book integrates language development with real-life projects, listening activities, pronunciation practice, and supplemental exercises, offering a practical approach to communication skills at home, work, and in daily routines.",
    50909: "A behind-the-scenes look at Roxanne Pulitzer’s highly publicized marriage and divorce, exploring the glamorous and turbulent world of Palm Beach society and the personal struggles that shaped her story.",
    54301: "An instructional guide to bowling that covers essential skills, techniques, and strategies. Includes explanations of rules, equipment, approach mechanics, and tips for improving performance.",
    57769: "A faith-centered guide encouraging readers to discover God’s purpose in their lives and participate more fully in spiritual revival. The book emphasizes personal vision, ministry involvement, and renewed commitment.",
    60608: "A creative reference for artists and hobbyists featuring decorative painting methods, color combinations, and project ideas. Includes inspiration from various artists and practical exercises.",
    61617: "A retelling of a Dominican legend about the ciguapas—mysterious sea-dwelling beings who visit land only at night. The story follows Guapa, a curious ciguapa whose love for sweet foods nearly exposes her tribe’s secret: their backward-facing feet that protect them from humans. A magical tale blending folklore, adventure, and cultural heritage.",
    65352: "A moving novel about Don Júbilo, a man gifted with the ability to understand the hidden emotions behind people’s words. Drawing from the life of Laura Esquivel’s father, the story explores communication, love, marriage, and the healing power of listening as past wounds threaten to break a family apart.",
    65897: "A devotional for women offering weekly reflections on faith, encouragement, and spiritual confidence. Through fifty-two short chapters, readers are reminded of God’s love, protection, and guidance through life’s challenges. Includes a full Spanish-language text in this edition."

}


In [ ]:
# Manual translation/correction to english.
# 2) Original dataframe update
for index, correction in manual_desc_updates_lang.items():
    if index in data_df.index:
        data_df.loc[index, 'Description'] = correction

In [ ]:
# Low confidence en texts
suspicious_en = data_lang[(data_lang['lang'] == 'en') & (data_lang['lang_conf'] < 0.8)]
suspicious_en[['Title', 'Description', 'lang_conf']].head(10)
#no change needed

In [ ]:
#data_df.to_csv("/content/processed_data_2.csv", index=False)

## KeyBERT

In [ ]:
data_df = pd.read_csv("/content/processed_data_2.csv")

In [ ]:
!pip install keybert sentence-transformers tqdm

In [ ]:
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

model = SentenceTransformer("all-mpnet-base-v2", device="cuda")
kw_model = KeyBERT(model=model)


In [ ]:
def get_topic_tags(description, n=5):
    if not isinstance(description, str) or len(description.strip()) < 10:
        return []

    keywords = kw_model.extract_keywords(
        description,
        keyphrase_ngram_range=(1, 2),
        stop_words='english',
        use_mmr=False,
        top_n=n
    )
    return [kw for kw, score in keywords]


In [ ]:
from tqdm.auto import tqdm

topic_tags = []

for desc in tqdm(data_df["Description"], desc="Extracting tags", ncols=120):
    topic_tags.append(get_topic_tags(desc))

data_df["Topic_Tags"] = topic_tags


In [ ]:
import pandas as pd
data_df = pd.read_csv("/content/processed_data_2.2.csv")

In [ ]:
data_df["Topic_Tags"].head()

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import ast # Import the ast module

all_keywords = []

# Convert string representations of lists to actual lists
data_df["Topic_Tags"] = data_df["Topic_Tags"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

for tags in data_df["Topic_Tags"]:
    if isinstance(tags, list):
        all_keywords.extend(tags)

freq = Counter(all_keywords)
top_n = 30
df_freq = pd.DataFrame(freq.most_common(top_n), columns=["keyword", "count"])

plt.figure(figsize=(10,12))
plt.barh(df_freq["keyword"], df_freq["count"])
plt.gca().invert_yaxis()
plt.title("Top 30 Most Common Topics")
plt.show()

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

model = SentenceTransformer('all-mpnet-base-v2')

unique_keywords = list(set(all_keywords))
emb = model.encode(unique_keywords, show_progress_bar=True)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score
import numpy as np
from tqdm import tqdm

def evaluate_kmeans_k(emb, k_values):
    results = {
        "k": [],
        "CH": [],
        "DBI": [],
        "Inertia": []
    }

    for k in tqdm(k_values):
        km = MiniBatchKMeans(
            n_clusters=k,
            random_state=0,
            batch_size=1024,
            n_init=5,
            max_iter=50
        )
        labels = km.fit_predict(emb)

        results["k"].append(k)
        results["CH"].append(calinski_harabasz_score(emb, labels))
        results["DBI"].append(davies_bouldin_score(emb, labels))
        results["Inertia"].append(km.inertia_)

    return results

k_values = list(range(20, 201, 20))
results = evaluate_kmeans_k(emb, k_values)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 10))

plt.subplot(3,1,1)
plt.plot(results["k"], results["CH"], marker="o")
plt.title("Calinski-Harabasz (Higher = Better)")
plt.grid(True)

plt.subplot(3,1,2)
plt.plot(results["k"], results["DBI"], marker="o")
plt.title("Davies-Bouldin (Lower = Better)")
plt.grid(True)

plt.subplot(3,1,3)
plt.plot(results["k"], results["Inertia"], marker="o")
plt.title("Inertia (Elbow Method — Lower is Better, look for elbow)")
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.cluster import MiniBatchKMeans

best_k = 80 #based on the visual output
km = MiniBatchKMeans(
    n_clusters=best_k,
    random_state=0,
    batch_size=1024,
    n_init=10
)

labels_best = km.fit_predict(emb)

clusters_best = {}
for kw, label in zip(unique_keywords, labels_best):
    clusters_best.setdefault(label, []).append(kw)


In [ ]:
# Show top 10 keywords from each cluster
for cluster_id, words in clusters_best.items():
    print(f"\n--- Cluster {cluster_id} ({len(words)} keywords) ---")
    print(", ".join(words[:10]))

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 1) Extract the centroids from KMeans
centroids = km.cluster_centers_   # shape: (num_clusters, embedding_dim)

# Number of clusters (K)
K = centroids.shape[0]

# 2) PCA on only the centroids
pca = PCA(n_components=2)
centroids_2d = pca.fit_transform(centroids)   # shape: (K, 2)

# 3) Optional: pick a few representative keywords per cluster
cluster_examples = {}
for kw, label in zip(unique_keywords, labels_best):
    if label not in cluster_examples:
        cluster_examples[label] = []
    if len(cluster_examples[label]) < 1:
        cluster_examples[label].append(kw)

# 4) Plot the cluster centroids
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 10))
plt.scatter(
    centroids_2d[:, 0],
    centroids_2d[:, 1],
    s=180,
    alpha=0.8,
    edgecolor="black"
)

for i, (x, y) in enumerate(centroids_2d):
    samples = ", ".join(cluster_examples.get(i, []))
    plt.text(
        x, y,
        f"{i}\n{samples}",
        fontsize=8,
        ha="center",
        va="center",
        bbox=dict(facecolor="white", alpha=0.2, edgecolor="none")
    )

plt.title("Keyword Cluster Centroids — PCA Projection", fontsize=16)
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid(alpha=0.3)
plt.show()



In [ ]:
#data_df.to_csv("/content/processed_data_2.2.csv", index=False)

## BERTopic

In [ ]:
!pip install -q bertopic sentence-transformers umap-learn hdbscan plotly tqdm

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap.umap_ as UMAP
import hdbscan
from tqdm.auto import tqdm
import numpy as np
import plotly.express as px

descriptions = data_df["Description"].fillna("").astype(str).tolist()
len(descriptions)
embedding_model = SentenceTransformer("all-mpnet-base-v2")


In [ ]:
umap_model = UMAP.UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=25,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)


In [ ]:
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    nr_topics="auto",   # merges similar topics
    verbose=True
)


In [ ]:
topics, probs = topic_model.fit_transform(descriptions)


In [ ]:
data_df["Topic"] = topics
data_df["Topic_Confidence"] = probs.max(axis=1)

In [ ]:
topic_model.save("bertopic_books_model")

In [ ]:
topic_model.get_topic_info().head(50)

In [ ]:
import random

# get topic metadata
topic_info = topic_model.get_topic_info()
# filter out outlier topic -1 (outliers)
valid_topics = topic_info[topic_info["Topic"] != -1]["Topic"].tolist()
# 10 random topics
sample_topics = random.sample(valid_topics, 10)

for topic_id in sample_topics:
    name = topic_info.loc[topic_info["Topic"] == topic_id, "Name"].values[0]
    print("\n" + "="*70)
    print(f"Topic {topic_id}:  {name}")
    print("="*70)

    # subset of books in this topic
    subset = data_df[data_df["Topic"] == topic_id].head(10)

    for title in subset["Title"].tolist():
        print("-", title)


In [ ]:
#10 words with highest importance for topic
topic_model.get_topic(-1)

In [ ]:
#topic_model.save("bertopic_books_model")

In [ ]:
def get_topic_label(topic_id, top_n=3):
    """Return a short label made of top words."""
    words = topic_model.get_topic(topic_id)
    if words is None:
        return "outlier"
    return "_".join([w for w, _ in words[:top_n]])

# Create label column
data_df["Topic_Label"] = data_df["Topic"].apply(get_topic_label)


In [ ]:
data_df.head()

In [ ]:
#data_df.to_csv("processed_data_2.3.csv", index=False)

In [ ]:
data_df = pd.read_csv("processed_data_2.3.csv")

## Generating Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from tqdm import tqdm

# Check GPU
print("Available device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Load embedding model on GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device=device)

In [ ]:
model = model.to("cuda")

Each books prediction is enriched with topic labels from BERTopic & extracted keywords from KeyBERT.

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model = SentenceTransformer("BAAI/bge-large-en-v1.5")
model = model.to(device)

def combine_text(row):
    base = row["Description"]
    kws = " ".join(row["Topic_Tags"]) if "Topic_Tags" in row else ""
    topic = row["Topic_Label"]
    return f"{base} Keywords: {kws} Topic: {topic}"

data_df["Final_Text"] = data_df.apply(combine_text, axis=1)

embeddings = model.encode(
    data_df["Final_Text"].tolist(),
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True,
    device=device
)

np.save("book_embeddings.npy", embeddings)
print("Saved book_embeddings.npy successfully")


In [ ]:
!pip install faiss-cpu

## Building the Semantic Search Index with FAISS

In [ ]:
import faiss
import numpy as np

dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)  # Cosine similarity (normalized vectors)
index.add(embeddings)

faiss.write_index(index, "book_faiss.index")


## Importing saved embeddings and faiss index

In [ ]:
import faiss
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer

# Load embeddings
embeddings = np.load("book_embeddings.npy")
# Load FAISS index
index = faiss.read_index("book_faiss.index")
# Load Dataset
data_df = pd.read_csv("processed_data_2.3.csv")
# query embedding model
query_model = SentenceTransformer("BAAI/bge-large-en-v1.5")

In [ ]:
print(embeddings.shape)
print(index.ntotal)

## Testing book queries

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

def search_books(query, index, data_df, top_k=10):
    # 1. Embed query
    q_emb = query_model.encode(
        [query],
        normalize_embeddings=True,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    # 2. FAISS search
    scores, ids = index.search(q_emb, top_k)

    # 3. results DataFrame
    result_rows = []
    for score, idx in zip(scores[0], ids[0]):
        result_rows.append({
            "Book_ID": int(idx),
            "Title": data_df.iloc[idx]["Title"],
            "Authors": data_df.iloc[idx]["Authors"],
            "Description": data_df.iloc[idx]["Description"],
            "Topic": data_df.iloc[idx].get("Topic_Label", None),
            "Score": float(score)
        })
    return pd.DataFrame(result_rows)

In [ ]:
pd.set_option('display.max_colwidth', None)
result = search_books("a book about nationalism and bravery", index, data_df, top_k=10)
result["Description"]
#"a cozy magical story about friendship and grief", "a book about nationalism and bravery"

Out of the top 10 returned books:

- 7/10 were highly relevant, matching at least 3 of the 4 target themes
(cozy, magical, friendship, grief)
- 2/10 were moderately relevant (friendship + emotional themes, but not magical)
- 1/10 was an outlier (non-fiction friendship gift book)

In [ ]:
"""from google.colab import files
files.download("book_embeddings.npy")"""